In [1]:
import coiled
import dask
import dask.dataframe as dd
import dask.distributed
import pandas as pd

In [2]:
cluster = coiled.Cluster(name="filter-timestamps", n_workers=5)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/rich/live.py:227: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


Found software environment build
Created FW rules: coiled-dask-matthew24-52301-firewall
Created scheduler VM: coiled-dask-matthew24-52301-scheduler (type: t3.medium, ip: ['3.228.23.71'])


In [6]:
pd.Timestamp("2010-01-01", tz="UTC")

Timestamp('2010-01-01 00:00:00+0000', tz='UTC')

In [10]:
pd.Timestamp("2010-01-01", tz="UTC").value

1262304000000000000

In [3]:
client = dask.distributed.Client(cluster)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/distributed/client.py:1105: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+-----------+
| Package     | client    | scheduler | workers   |
+-------------+-----------+-----------+-----------+
| cloudpickle | 1.6.0     | 2.0.0     | 2.0.0     |
| dask        | 2021.08.1 | 2021.09.1 | 2021.09.1 |
| distributed | 2021.08.1 | 2021.09.1 | 2021.09.1 |
| pandas      | 1.3.2     | 1.3.3     | 1.3.3     |
+-------------+-----------+-----------+-----------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [11]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
    filters=[
        [
            ("id", ">", pd.Timestamp("2010-01-01", tz="UTC").value),
            ("id", "<", pd.Timestamp("2010-02-01", tz="UTC").value),
        ]
    ]
)

In [12]:
ddf.head()

,id,name,x,y
timestamp,,,,


In [13]:
len(ddf)

0

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [ ]:
ddf = dd.read_parquet(
    path="s3://[BUCKET_NAME]/[FOLDER/PREFIX]/*",
    columns=["COL1", "COL2", "DATE_COL"],
    filters=[
        [
            ("DATE_COL", ">=", pd.Timestamp("2020-05-01")),
            ("DATE_COL", "<", pd.Timestamp("2020-06-01")),
        ]
    ],
    engine="pyarrow-dataset",
)